In [23]:
from flask import Flask, Response, request, jsonify, render_template, url_for
from utils import movies, lookup_movie, movie_average_rating, genres, neighbor_recommender, movies_genres_ohe
from recommender import recommend_most_popular, recommend_random

import pandas as pd

In [2]:
user = {'ratings': {'arielle': '5', 'lion king': '5', 'toy story': '4', 'jungle book': '5'}, 'genres': ['musical', 'children', 'comedy']}

In [3]:
user

{'ratings': {'arielle': '5',
  'lion king': '5',
  'toy story': '4',
  'jungle book': '5'},
 'genres': ['musical', 'children', 'comedy']}

In [4]:
neighbor_recommender(user)

['Beauty and the Beast (1991)',
 'Aladdin (1992)',
 'Shawshank Redemption, The (1994)',
 'Fugitive, The (1993)',
 'Dances with Wolves (1990)',
 'Apollo 13 (1995)',
 'Batman (1989)',
 'Ace Ventura: Pet Detective (1994)',
 'Babe (1995)',
 'Braveheart (1995)']

In [5]:
lookup_movie('Lion King', movies['title'])

[('Lion King, The (1994)', 90, 364),
 ("Lion King II: Simba's Pride, The (1998)", 90, 26999),
 ('Lion King 1½, The (2004)', 90, 27619),
 ("Kid in King Arthur's Court, A (1995)", 86, 258),
 ('Madness of King George, The (1994)', 86, 272)]

In [84]:
filtered_dfs = {}
for genre in user['genres']:
    filtered_dfs[genre]=movies_genres_ohe[movies_genres_ohe[genre] ==1]

    

In [98]:
df = pd.DataFrame(columns=movies_genres_ohe.columns)
for genre in user['genres']:
    df = pd.concat([df, movies_genres_ohe[movies_genres_ohe[genre] ==1]])
df.drop_duplicates(inplace=True)

In [100]:
df.shape

(4175, 21)

In [73]:
filter1 = (movies_genres_ohe['musical'] ==1)
filter2 = (movies_genres_ohe['children'] ==1)
filter3 = (movies_genres_ohe['comedy'] ==1)
musical_movies = movies_genres_ohe[filter1]
children_movies = movies_genres_ohe[filter2]
comedy_movies = movies_genres_ohe[filter3]

In [74]:
comedy_movies.loc[144606]

title          Confessions of a Dangerous Mind (2002)
genres            Comedy|Crime|Drama|Romance|Thriller
romance                                             1
sci-fi                                              0
animation                                           0
film-noir                                           0
musical                                             0
adventure                                           0
thriller                                            1
horror                                              0
documentary                                         0
fantasy                                             0
mystery                                             0
children                                            0
comedy                                              1
crime                                               1
western                                             0
imax                                                0
war                         

In [75]:
genre_movies = pd.concat([musical_movies,children_movies,comedy_movies], axis=0)
genre_movies.drop_duplicates(inplace=True, keep='first')

In [76]:
genre_movies.shape

(4175, 21)

In [78]:
movies_genres_ohe_filtered = movies_genres_ohe[filter1|filter2|filter3]

In [79]:
movies.loc[genre_movies.index]

,title,genres
48,Pocahontas (1995),Animation|Children|Drama|Musical|Romance
107,Muppet Treasure Island (1996),Adventure|Children|Comedy|Musical
199,"Umbrellas of Cherbourg, The (Parapluies de Che...",Drama|Musical|Romance
242,Farinelli: il castrato (1994),Drama|Musical
346,Backbeat (1993),Drama|Musical
...,...,...
193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi
193571,Silver Spoon (2014),Comedy|Drama
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy


In [48]:
filter4= ~movies_genres_ohe_filtered.index.isin(genre_movies.index)

In [49]:
movies_genres_ohe_filtered[filter4]

,title,genres,romance,sci-fi,animation,film-noir,musical,adventure,thriller,horror,...,fantasy,mystery,children,comedy,crime,western,imax,war,drama,action
144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,1,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0


In [72]:
movies_genres_ohe[movies_genres_ohe['title']=='Confessions of a Dangerous Mind (2002)']

,title,genres,romance,sci-fi,animation,film-noir,musical,adventure,thriller,horror,...,fantasy,mystery,children,comedy,crime,western,imax,war,drama,action
6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Thriller,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,1,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
